# Financial Policy at Apple

FINM 35500

Spring 2022

Mark Hendricks

**This notebook was presented in Class Session 3, but it has been lightly edited for clarity since then. Thus, it will not exactly match the version seen in class.**


## The Situation

Case is in 2012.

Apple (AAPL) is coming off a transformative period, (iPhone, iPad, Jobs passing.)
- Profitability is high.
- Cash and liquid securiites have grown huge on balance sheet. 
- Productive use for this capital?
- If not, return to shareholders via dividend, repurchases?

Activist investors like Einhorn believe the cash reserves are hurting investor returns.
- Believes that it is excess.
- Believes investors are ignoring it--would get same P/E ratio without it.
- Spin it off in preferred shares, and boost total value 33\% through engineering.

# Load Financial Statements

In [1]:
import numpy as np
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
from scipy.optimize import fsolve
from matplotlib import pyplot as plt

In [2]:
path_load_exhibits = '../data/exhibits_apple.xlsx'
summary = pd.read_excel(path_load_exhibits,sheet_name='Annual Summary',index_col=0,skiprows=1)
summary.dropna(axis=0,how='any',inplace=True)
summary

,2000-09-30,2001-09-29,2002-09-28,2003-09-27,2004-09-25,2005-09-24,2006-09-30,2007-09-29,2008-09-27,2009-09-26,2010-09-25,2011-09-24,2012-09-29
For the Fiscal Year Ending,,,,,,,,,,,,,
Total Revenue,"7,983.00","5,363.00","5,742.00","6,207.00","8,279.00","13,931.00","19,315.00","24,578.00","37,491.00","42,905.00","65,225.00","108,249.00","156,508.00"
Cost of Goods Sold,"5,817.00","4,128.00","4,139.00","4,499.00","6,022.00","9,889.00","13,717.00","16,426.00","24,294.00","25,683.00","39,541.00","64,431.00","87,846.00"
"Sales, General, and Admin.","1,256.00","1,138.00","1,109.00","1,212.00","1,430.00","1,864.00","2,433.00","2,963.00","3,761.00","4,149.00","5,517.00","7,599.00","10,040.00"
Gross Profit,"2,166.00","1,235.00","1,603.00","1,708.00","2,257.00","4,042.00","5,598.00","8,152.00","13,197.00","17,222.00","25,684.00","43,818.00","68,662.00"
EBIT,530.00,-333.00,48.00,25.00,336.00,"1,643.00","2,453.00","4,407.00","8,327.00","11,740.00","18,385.00","33,790.00","55,241.00"
Net Income,786.00,-25.00,65.00,69.00,266.00,"1,328.00","1,989.00","3,495.00","6,119.00","8,235.00","14,013.00","25,922.00","41,733.00"
Cash Flow,868.00,185.00,89.00,289.00,934.00,"2,535.00","2,220.00","5,470.00","9,596.00","10,159.00","18,595.00","37,529.00","50,856.00"
Cash and Marketable Securities,"4,027.00","4,336.00","4,337.00","4,566.00","5,464.00","8,261.00","10,110.00","15,386.00","24,490.00","33,992.00","51,011.00","81,570.00","121,251.00"
Accounts Receivable,953.00,466.00,565.00,766.00,774.00,895.00,"1,252.00","1,637.00","2,422.00","3,361.00","5,510.00","5,369.00","10,930.00"


In [3]:
balance = pd.read_excel(path_load_exhibits,sheet_name='Balance Sheet',index_col=0,skiprows=1)
balance.dropna(axis=0,how='any',inplace=True)
balance.index = balance.index.str.strip()
balance.columns = pd.to_datetime(balance.columns)
balance

,2010-09-25,2011-09-24,2012-09-29
Balance Sheet as of:,,,
Cash And Equivalents,11261,9815,10746
Short Term Investments,14359,16137,18383
Trading Asset Securities,-,-,-
Total Cash & ST Investments,25620,25952,29129
Accounts Receivable,5510,5369,10930
Other Receivables,4414,6348,7762
Total Receivables,9924,11717,18692
Inventory,1051,776,791
"Deferred Tax Assets, Curr.",1636,2014,2583


In [4]:
income = pd.read_excel(path_load_exhibits,sheet_name='Income Statement',index_col=0,skiprows=1)
income.dropna(axis=0,how='any',inplace=True)
income.index = income.index.str.strip()
income.columns = pd.to_datetime(income.columns)
income

,2010-09-25,2011-09-24,2012-09-29
For the Fiscal Year Ending,,,
Revenue,65225,108249,156508
Cost of Goods Sold,39541,64431,87846
Gross Profit,25684,43818,68662
"Sales, General, and Administration Expenses",5517,7599,10040
Research and Development Expenses,1782,2429,3381
Total Operating Expenses,7299,10028,13421
Operating Income,18385,33790,55241
Interest Expense,-,-,-
Interest and Investment Income,311,519,1088


In [5]:
cashstate = pd.read_excel(path_load_exhibits,sheet_name='Cash Flows',index_col=0,skiprows=1)
cashstate.dropna(axis=0,how='any',inplace=True)
cashstate.index = cashstate.index.str.strip()
cashstate.columns = pd.to_datetime(cashstate.columns)
cashstate

,2010-09-25,2011-09-24,2012-09-29
For the Fiscal Year Ending,,,
Net Income,14013,25922,41733
Depreciation & Amortization,958,1622,2672
Amortization of Goodwill and Intangibles,69,192,605
Total Depreciation & Amortization,1027,1814,3277
Stock-Based Compensation,879,1168,1740
Other Operating Activities,1440,2868,4405
Change in Accounts Receivable,-2142,143,-5551
Change In Inventories,-596,275,-15
Change in Accounts Payable,6307,2515,4467


# Policy in 2013 (Section 1)

## Cash (1.3a)

- repatriated
- required
- excess

In [6]:
FRACTION_CASH_UNTAXED = .69
TAX_RATE_REPATRIATED_CASH = .35

total_cash = summary.loc['Cash and Marketable Securities']

repatriation_tax = FRACTION_CASH_UNTAXED * TAX_RATE_REPATRIATED_CASH * total_cash
repatriated_cash = total_cash - repatriation_tax
required_cash = 2 * summary.loc['Sales, General, and Admin.']

excess_cash_repatriated = repatriated_cash - required_cash
excess_cash_untaxed = total_cash - required_cash

excess_cash = pd.concat([excess_cash_repatriated,excess_cash_untaxed],keys=['repatriated','untaxed'],axis=1).T
excess_cash.name = 'excess cash'
excess_cash

,2000-09-30,2001-09-29,2002-09-28,2003-09-27,2004-09-25,2005-09-24,2006-09-30,2007-09-29,2008-09-27,2009-09-26,2010-09-25,2011-09-24,2012-09-29
repatriated,542.48,"1,012.86","1,071.61","1,039.31","1,284.44","2,537.97","2,802.44","5,744.28","11,053.67","17,484.93","27,657.84","46,672.85","71,888.88"
untaxed,"1,515.00","2,060.00","2,119.00","2,142.00","2,604.00","4,533.00","5,244.00","9,460.00","16,968.00","25,694.00","39,977.00","66,372.00","101,171.00"


## Net Working Capital (1.2)

We analyze NWC and its components.

### Difference if using Annual Summary vs Balance Sheet
The Annual Summary gives a longer time-series, but it leaves out some components of current assets and current liabilities.

The Balance Sheet data includes a substantial amount of current assets and current liabilities, but it only provides 3 years of data.

If using the annual summary data, it is important to subtract out excess cash. Much of the cash listed in the Annual Summary is not "current" but rather long-term securities.

Note that the difference between using the Annual Summary and Balance Sheet only impacts the NWC numbers--not the inventory-days, AR-days, nor AP-days.

In [7]:
nwc = pd.DataFrame({'NWC (Annual Summary)': summary.loc['Accounts Receivable'] - summary.loc['Accounts Payable'] + summary.loc['Inventory'] + required_cash}).T
nwc.loc['NWC (Balance Sheet)'] = balance.loc['Total Current Assets'] - balance.loc['Total Current Liabilities']
nwc.style.format("{:,.1f}")

,2000-09-30 00:00:00,2001-09-29 00:00:00,2002-09-28 00:00:00,2003-09-27 00:00:00,2004-09-25 00:00:00,2005-09-24 00:00:00,2006-09-30 00:00:00,2007-09-29 00:00:00,2008-09-27 00:00:00,2009-09-26 00:00:00,2010-09-25 00:00:00,2011-09-24 00:00:00,2012-09-29 00:00:00
NWC (Annual Summary),"2,341.0","1,952.0","1,917.0","2,092.0","2,284.0","3,009.0","2,998.0","2,939.0","4,933.0","6,513.0","5,580.0","6,711.0","10,626.0"
NWC (Balance Sheet),nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,"20,956.0","17,018.0","19,111.0"


In [8]:
nwc_assets = nwc / summary.loc['Total Assets']
nwc_revenues = nwc / summary.loc['Total Revenue']

nwc_comp = pd.concat([nwc_assets,nwc_revenues],axis=0,keys=['NWC-to-Assets','NWC-to-Revenue'])
nwc_comp.style.format("{:.1%}")

#### End of Period CCC

The Cash Conversion Cycle is using balance sheet data and income statement data. 

This raises the issue of whether to use start, end, or mid-period values from the balance sheet. 

For this analysis, we simply use end-of-period balance-sheet data, but taking an average of beginning-and-ending values is also typical.

In [9]:
nwc_days = pd.DataFrame({'Inventory Days': 365 * summary.loc['Inventory'] / summary.loc['Cost of Goods Sold']}).T 
nwc_days.loc['Accounts Receivable Days'] = 365 * summary.loc['Accounts Receivable'] / summary.loc['Total Revenue']
nwc_days.loc['Accounts Payable Days'] = 365 * summary.loc['Accounts Payable'] / summary.loc['Cost of Goods Sold'] 

nwc_days.loc['Cash Conversion Cycle'] = nwc_days.loc['Inventory Days'] + nwc_days.loc['Accounts Receivable Days'] - nwc_days.loc['Accounts Payable Days']

nwc_days.style.format("{:.1f}")

,2000-09-30 00:00:00,2001-09-29 00:00:00,2002-09-28 00:00:00,2003-09-27 00:00:00,2004-09-25 00:00:00,2005-09-24 00:00:00,2006-09-30 00:00:00,2007-09-29 00:00:00,2008-09-27 00:00:00,2009-09-26 00:00:00,2010-09-25 00:00:00,2011-09-24 00:00:00,2012-09-29 00:00:00
Inventory Days,2.1,1.0,4.0,4.5,6.1,6.1,7.2,7.7,7.6,6.5,9.7,4.4,3.3
Accounts Receivable Days,43.6,31.7,35.9,45.0,34.1,23.4,23.7,24.3,23.6,28.6,30.8,18.1,25.5
Accounts Payable Days,72.6,70.8,80.3,93.6,87.9,65.7,90.2,110.4,82.9,79.6,110.9,82.9,88.0
Cash Conversion Cycle,-27.0,-38.1,-40.5,-44.0,-47.7,-36.1,-59.4,-78.4,-51.7,-44.5,-70.4,-60.4,-59.2


## Analyzing ROE (1.1 and 1.3b)

A few things to note about DuPont analysis of ROE below:

Recall that it decomposes ROE into three factors:

$\text{ROE} = \underbrace{\frac{\text{Net Income}}{\text{Revenues}}}_{\text{Profit Margin}} \times \underbrace{\frac{\text{Revenues}}{\text{Book Assets}}}_{\text{Asset Turnover}} \times \underbrace{\frac{\text{Book Assets}}{\text{Book Equity}}}_{\text{DuPont Leverage}}$

The last factor is labeled "DuPont Leverage" to clarify that it is a particular measure of leverage, but it is not the typical measure of leverage. (It includes all assets versus equity instead of debt versus equity.) It is used in DuPont analysis to allow for the desired decomposition of ROE.

### Excluding Excess Cash

The asset turnover and DuPont leverage metrics change substantially if Apple were to have not carried any excess cash (or its associated repatriatiation tax) during this time.

Below, we calculate the DuPont analysis for the stated financials and for the hypothetical version without excess cash.

(In the hypothetical, the excess cash was distributed, and book equity goes down accordingly.)

In [10]:
capital_historic = pd.DataFrame({'assets': summary.loc['Total Assets']}).T
capital_historic.loc['book equity'] = summary.loc['Total Shareholders\' Equity']
capital_historic.loc['debt'] = summary.loc['Long-term Debt']

capital_historic = pd.concat([capital_historic, capital_historic - excess_cash.loc['untaxed']],keys=['stated','removing excess cash'])
capital_historic.loc['removing excess cash','debt'] += excess_cash.loc['untaxed']
capital_historic

2000-09-30  2001-09-29  2002-09-28  \
stated               assets         6,803.00    6,021.00    6,298.00   
                     book equity    4,107.00    3,920.00    4,095.00   
                     debt             300.00      317.00      316.00   
removing excess cash assets         5,288.00    3,961.00    4,179.00   
                     book equity    2,592.00    1,860.00    1,976.00   
                     debt             300.00      317.00      316.00   

                                  2003-09-27  2004-09-25  2005-09-24  \
stated               assets         6,815.00    8,050.00   11,516.00   
                     book equity    4,223.00    5,076.00    7,428.00   
                     debt               0.00        0.00        0.00   
removing excess cash assets         4,673.00    5,446.00    6,983.00   
                     book equity    2,081.00    2,472.00    2,895.00   
                     debt               0.00        0.00        0.00   

                                  2006-09-30  2007-09-29  2008-09-27  \
stated               assets        17,205.00   25,347.00   36,171.00   
                     book equity    9,984.00   14,532.00   22,297.00   
                     debt               0.00        0.00        0.00   
removing excess cash assets        11,961.00   15,887.00   19,203.00   
                     book equity    4,740.00    5,072.00    5,329.00   
                     debt               0.00        0.00        0.00   

                                  2009-09-26  2010-09-25  2011-09-24  \
stated               assets        47,501.00   75,183.00  116,371.00   
                     book equity   31,640.00   47,791.00   76,615.00   
                     debt               0.00        0.00        0.00   
removing excess cash assets        21,807.00   35,206.00   49,999.00   
                     book equity    5,946.00    7,814.00   10,243.00   
                     debt               0.00        0.00        0.00   

                                  2012-09-29  
stated               assets       176,064.00  
                     book equity  118,210.00  
                     debt               0.00  
removing excess cash assets        74,893.00  
                     book equity   17,039.00  
                     debt               0.00

In [11]:
roe = pd.DataFrame({'Earnings-to-EBIT':summary.loc['Net Income'] / summary.loc['EBIT']}).T
roe.loc['Gross Margin'] = summary.loc['EBIT']/summary.loc['Total Revenue']

roe.loc['Asset Turnover'] = summary.loc['Total Revenue']/capital_historic.loc['stated','assets']
roe.loc['DuPont Leverage'] = capital_historic.loc['stated','assets'] / capital_historic.loc['stated','book equity']
roe.loc['ROE'] = summary.loc['Net Income'] / capital_historic.loc['stated','book equity']

In [12]:
roe_invested = roe.loc[['Earnings-to-EBIT','Gross Margin']]

roe_invested.loc['Asset Turnover'] = summary.loc['Total Revenue']/ capital_historic.loc['removing excess cash','assets']
roe_invested.loc['DuPont Leverage'] = capital_historic.loc['removing excess cash','assets'] / capital_historic.loc['removing excess cash','book equity']
roe_invested.loc['ROE'] = summary.loc['Net Income'] / capital_historic.loc['removing excess cash','book equity']

roe_all = pd.concat([roe,roe_invested],axis=0,keys=['Total Assets','Invested Capital'])
roe_all.style.format("{:.1%}")

In either version (with or without excess cash included,) we see the following:
- The percentage of operating income (EBIT) converted to earnings is half what it was in 2000.
- Gross Margin has gone up massively since the release of the iPhone in 2007.
- Asset turnover has not changed too much, though a bit on the low end at the end of the period. But if we looked at operating assets, this wouldn't be true. This measure of asset turnover is including massive amounts of (non-operating) excess cash.
- Book leverage has not changed much, though again, it is including non-operating assets (excess cash) and non-debt liabilities.

Now consider the difference when using Invested Capital (no excess cash)
- ROE in 2012 is 35% when the excess cash is balanced by book equity
- ROE in 2012 is 245% when the excess cash (and associated repatriation tax) is removed

## Forecasting 2012-2017 (1.4)

- We are not forecasting any non-operating income.
- Sales, COGS, SG&A are each forecast to grow proportionally, so we can directly forecast EBIT to grow at this rate.

See the excel book, `apple_discussion.xlsx`.

---
---

# Analyzing Apple at the End of 2021

Financial Statements come from WRDS.

#### The statements have two discrepancies:

1. **Depreciation** on the income statement (and cashflow statement) do not match the change in accumulated deprecation on the balance sheet. 
    
    The former is substantially larger than the latter. There are a few potential explanations for this.
    - Firms may sell assets and their corresponding accumulated depreciation comes off the balance sheet, but the expensed depreciation has already impacted the income statement.
    - Firms may be including depreciation and amortization from other accounts in the expensed depreciation line, (such as depreciation to deferred taxes.)

   For Apple, it is not clear what explains the discrepancy. In any case, ignore it, and use the depreciation amount on the income statement (and cashflow statement.)
   
   
2. **CapEx** reported in the cashflow statement does not reconcile to the PP&E accounts on the balance sheet.

    $\text{CAPEX}_t = \text{Gross PPE}_t - \text{Gross PPE}_{t-1} = \text{Net PPE}_t - \text{Net PPE}_{t-1} + \text{Depreciation}_t$

    But this does not reconcile. There are a few possibilities, similar to the potential explanations for mismatched depreciation above.
    
    If there is a mismatch, typically use the cashflow statement CapEx rather than the imputed CapEx from the PP&E account. The accountants that compiled the CapEx statement should have had access to more detailed information in making this calculation.

## Load Data

A few data cleaning issue
- There are duplicate rows, as some show up in income statement and cashflow statement, etc.
- Some fields have `.` instead of `0`.

In [13]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [14]:
path_load_exhibits = '../data/financials_apple_2022.xlsx'
financials = pd.read_excel(path_load_exhibits,sheet_name='Financial Statement',index_col=0,skiprows=2)

financials.dropna(axis=0,how='any',inplace=True)

financials.index = financials.index.str.strip()
financials = financials[~financials.index.duplicated(keep='first')]

financials = financials.replace({'.':'0'}).astype(float)

financials

/Users/markhendricks/Projects/FINM/finm-corp-2022/venv/lib/python3.9/site-packages/openpyxl/worksheet/header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
Cash and Short-Term Investments,"4,027.00","4,336.00","4,337.00","4,566.00","5,464.00","8,261.00","10,110.00","15,386.00","24,490.00","23,464.00","25,620.00","25,952.00","29,129.00","40,546.00","25,077.00","41,601.00","67,155.00","74,181.00","66,301.00","100,580.00","90,979.00","62,639.00"
Cash,"1,191.00","2,310.00","2,252.00","3,396.00","2,969.00","3,491.00","6,392.00","9,352.00","11,875.00","5,263.00","11,261.00","9,815.00","10,746.00","14,259.00","13,844.00","21,120.00","20,484.00","20,289.00","25,913.00","48,844.00","38,016.00","34,940.00"
Short-Term Investments,"2,836.00","2,026.00","2,085.00","1,170.00","2,495.00","4,770.00","3,718.00","6,034.00","12,615.00","18,201.00","14,359.00","16,137.00","18,383.00","26,287.00","11,233.00","20,481.00","46,671.00","53,892.00","40,388.00","51,736.00","52,963.00","27,699.00"
Receivables,953.00,466.00,565.00,950.00,"1,050.00","1,312.00","2,845.00","4,029.00","4,704.00","5,057.00","9,924.00","11,717.00","18,692.00","20,641.00","27,219.00","30,343.00","29,299.00","35,673.00","48,995.00","45,804.00","37,445.00","51,506.00"
Receivables - Trade,953.00,466.00,565.00,766.00,774.00,895.00,"1,252.00","1,637.00","2,422.00","3,361.00","5,510.00","5,369.00","10,930.00","13,102.00","17,460.00","16,849.00","15,754.00","17,874.00","23,186.00","22,926.00","16,120.00","26,278.00"
Income Tax Refund,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Current Receivables - Other,0.00,0.00,0.00,184.00,276.00,417.00,"1,593.00","2,392.00","2,282.00","1,696.00","4,414.00","6,348.00","7,762.00","7,539.00","9,759.00","13,494.00","13,545.00","17,799.00","25,809.00","22,878.00","21,325.00","25,228.00"
Inventories - Total,33.00,11.00,45.00,56.00,101.00,165.00,270.00,346.00,509.00,455.00,"1,051.00",776.00,791.00,"1,764.00","2,111.00","2,349.00","2,132.00","4,855.00","3,956.00","4,106.00","4,061.00","6,580.00"
Inventories - Raw Materials,1.00,1.00,9.00,2.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,124.00,683.00,471.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
Inventories - Work in Process,2.00,0.00,0.00,4.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


## ROIC Analysis (2.1-2.2)

#### Taxes

For modeling historic taxes, one could use...
- a constant effective rate
- the actual taxes paid year-by-year for the historical data, and an imputed tax rate for the future.

#### NOP or EBIT?
Non-operating income is negligible in most years, so using NOP or EBIT is fine.
- Both are shown in the dataframe `profit` below, just for pedagogical reasons.

In [15]:
profit = pd.DataFrame({'nop': financials.loc['Operating Income After Depreciation']}).T

keys_ebit_adj = ['Nonoperating Income (Expense) - Excluding Interest Income','Special Items']
profit.loc['ebit'] = profit.loc['nop'] + financials.loc[keys_ebit_adj].sum()
profit.loc['tax'] = financials.loc['Income Taxes - Total']
profit.loc['nopat'] = profit.loc['nop'] - profit.loc['tax']
profit.loc['ebiat'] = profit.loc['ebit'] - profit.loc['tax']
profit.loc['net interest expense'] = financials.loc['Interest Expense'] - financials.loc['Interest Income']
profit

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
nop,620.00,-333.00,46.00,25.00,349.00,"1,650.00","2,453.00","4,409.00","6,275.00","11,740.00","18,385.00","33,790.00","55,241.00","48,999.00","52,503.00","71,230.00","59,476.00","61,344.00","70,662.00","63,930.00","66,288.00","108,949.00"
ebit,903.00,-254.00,-20.00,31.00,322.00,"1,632.00","2,424.00","4,361.00","6,242.00","11,659.00","18,229.00","34,205.00","55,763.00","50,291.00","53,867.00","73,248.00","62,828.00","66,412.00","76,143.00","69,313.00","69,964.00","111,852.00"
tax,306.00,-15.00,22.00,24.00,107.00,480.00,829.00,"1,512.00","2,061.00","3,831.00","4,527.00","8,283.00","14,030.00","13,118.00","13,973.00","19,121.00","15,685.00","15,738.00","13,372.00","10,481.00","9,680.00","14,527.00"
nopat,314.00,-318.00,24.00,1.00,242.00,"1,170.00","1,624.00","2,897.00","4,214.00","7,909.00","13,858.00","25,507.00","41,211.00","35,881.00","38,530.00","52,109.00","43,791.00","45,606.00","57,290.00","53,449.00","56,608.00","94,422.00"
ebiat,597.00,-239.00,-42.00,7.00,215.00,"1,152.00","1,595.00","2,849.00","4,181.00","7,828.00","13,702.00","25,922.00","41,733.00","37,173.00","39,894.00","54,127.00","47,143.00","50,674.00","62,771.00","58,832.00","60,284.00","97,325.00"
net interest expense,-189.00,-202.00,-107.00,-61.00,-61.00,-183.00,-394.00,-647.00,-653.00,-407.00,-311.00,0.00,0.00,136.00,384.00,733.00,"1,456.00","2,323.00","3,240.00","3,576.00","2,873.00","2,645.00"


### Excess Cash

The homework directed you to assume
$\text{current cash} = \text{required cash}$
for simplicity.

However, we also examine using a model for required cash--the same as was used in the Harvard Case.
$\text{required cash} = 2\times\text{SG&A}$

Set the parameter `USE_CURRENT_CASH_AS_REQUIRED`

In [16]:
USE_CURRENT_CASH_AS_REQUIRED = False

cash = pd.DataFrame({'current cash': financials.loc['Cash and Short-Term Investments']}).T
keys_excess_cash = ['Invest. & Advances - Eq.Meth.','Invest. & Advances - Other']
cash.loc['long-term cash'] = financials.loc[keys_excess_cash].sum()
cash.loc['total cash'] = cash.loc[['current cash','long-term cash']].sum()

if USE_CURRENT_CASH_AS_REQUIRED:
    cash.loc['required cash'] = cash.loc['current cash']
else:
    cash.loc['required cash'] = 2* financials.loc['Selling, General, and Administrative Expense']
    cash.loc['required cash'] = cash.loc[['required cash','total cash']].min()
    
cash.loc['excess cash'] = cash.loc['total cash'] - cash.loc['required cash']

cash

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
current cash,"4,027.00","4,336.00","4,337.00","4,566.00","5,464.00","8,261.00","10,110.00","15,386.00","24,490.00","23,464.00","25,620.00","25,952.00","29,129.00","40,546.00","25,077.00","41,601.00","67,155.00","74,181.00","66,301.00","100,580.00","90,979.00","62,639.00"
long-term cash,0.00,0.00,0.00,5.00,1.40,0.00,0.00,0.00,0.00,"10,528.00","25,391.00","55,618.00","92,122.00","106,215.00","130,162.00","164,065.00","170,430.00","194,714.00","170,799.00","105,341.00","100,887.00","127,877.00"
total cash,"4,027.00","4,336.00","4,337.00","4,571.00","5,465.40","8,261.00","10,110.00","15,386.00","24,490.00","33,992.00","51,011.00","81,570.00","121,251.00","146,761.00","155,239.00","205,666.00","237,585.00","268,895.00","237,100.00","205,921.00","191,866.00","190,516.00"
required cash,"3,092.00","3,136.00","3,114.00","3,366.00","3,820.00","4,786.00","6,290.00","7,490.00","9,740.00","10,964.00","14,598.00","20,056.00","26,842.00","30,610.00","36,068.00","44,792.00","48,478.00","53,684.00","61,882.00","68,924.00","77,336.00","87,774.00"
excess cash,935.00,"1,200.00","1,223.00","1,205.00","1,645.40","3,475.00","3,820.00","7,896.00","14,750.00","23,028.00","36,413.00","61,514.00","94,409.00","116,151.00","119,171.00","160,874.00","189,107.00","215,211.00","175,218.00","136,997.00","114,530.00","102,742.00"


### Capital

Consider 
- Debt
- Book Equity
- Gross Invested Capital = Debt + Book Equity

And 
- Net Debt = Debt - Excess Cash
- Invested Capital = Net Debt + Book Equity

In [17]:
keys_debt = ['Debt in Current Liabilities','Long-Term Debt - Total']
capital = pd.DataFrame({'debt': financials.loc[keys_debt].sum()}).T
capital.loc['net debt'] = capital.loc['debt'] - cash.loc['excess cash']
capital.loc['book equity'] = financials.loc['Shareholders Equity - Total']
capital.loc['gross invested capital'] = capital.loc['debt'] + capital.loc['book equity']
capital.loc['invested capital'] = capital.loc['net debt'] + capital.loc['book equity']
capital

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
debt,300.00,317.00,316.00,304.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"16,960.00","35,295.00","64,462.00","87,032.00","115,680.00","114,483.00","108,047.00","122,278.00","136,522.00"
net debt,-635.00,-883.00,-907.00,-901.00,"-1,645.40","-3,475.00","-3,820.00","-7,896.00","-14,750.00","-23,028.00","-36,413.00","-61,514.00","-94,409.00","-99,191.00","-83,876.00","-96,412.00","-102,075.00","-99,531.00","-60,735.00","-28,950.00","7,748.00","33,780.00"
book equity,"4,107.00","3,920.00","4,095.00","4,223.00","5,076.00","7,466.00","9,984.00","14,532.00","21,030.00","31,640.00","47,791.00","76,615.00","118,210.00","123,549.00","111,547.00","119,355.00","128,249.00","134,047.00","107,147.00","90,488.00","65,339.00","63,090.00"
gross invested capital,"4,407.00","4,237.00","4,411.00","4,527.00","5,076.00","7,466.00","9,984.00","14,532.00","21,030.00","31,640.00","47,791.00","76,615.00","118,210.00","140,509.00","146,842.00","183,817.00","215,281.00","249,727.00","221,630.00","198,535.00","187,617.00","199,612.00"
invested capital,"3,472.00","3,037.00","3,188.00","3,322.00","3,430.60","3,991.00","6,164.00","6,636.00","6,280.00","8,612.00","11,378.00","15,101.00","23,801.00","24,358.00","27,671.00","22,943.00","26,174.00","34,516.00","46,412.00","61,538.00","73,087.00","96,870.00"


### ROIC decomposed

Similar to a DuPont analysis, decompose ROIC as profit margin and asset turnover.

$\text{ROIC} \equiv \frac{\text{NOPAT}_t}{\text{Invested Capital}_{t-1}} = \underbrace{\frac{\text{NOPAT}_t}{\text{Revenue}_t}}_{\text{(NOPAT) Profit Margin}} \times \underbrace{\frac{\text{Revenue}_t}{\text{Invested Capital}_{t-1}}}_{\text{Turnover (Lagged IC)}}$

In [18]:
roic = pd.DataFrame({'roic':profit.loc['nopat'] / capital.loc['invested capital'].shift()}).T
roic.loc['profit margin'] = profit.loc['nopat'] / financials.loc['Sales (Net)']
roic.loc['turnover (invested capital)'] = financials.loc['Sales (Net)'] / capital.loc['invested capital'].shift()
roic

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
roic,NaN,-0.09,0.01,0.00,0.07,0.34,0.41,0.47,0.64,1.26,1.61,2.24,2.73,1.51,1.58,1.88,1.91,1.74,1.66,1.15,0.92,1.29
profit margin,0.04,-0.06,0.00,0.00,0.03,0.08,0.08,0.12,0.13,0.18,0.21,0.24,0.26,0.21,0.21,0.22,0.20,0.20,0.22,0.21,0.21,0.26
turnover (invested capital),NaN,1.54,1.89,1.95,2.49,4.06,4.84,3.89,4.89,6.83,7.57,9.51,10.36,7.18,7.50,8.45,9.38,8.76,7.69,5.61,4.46,5.01


### ROIC Conclusions

We see a few things since the case in 2013.

- ROIC jumped in 2011, and it has stayed elevated through the years since
- Same for profit margin
- Turnover has fluctuated substantially, but overall has been high since the case.

#### Lagged or Averaged Balance Sheet
ROIC is a flow (income statement) divided by a stock (balance sheet). Thus, one could compute it with the denominator
- same-period
- lagged
- average of the two

Above, we use lagged, as we do for the ROIC. One could certainly make a different choice for ROIC.

Whichever choice we make, we do the same for the Invested Capital Turnover, so that ROIC is properly decomposed into the two metrics.

### Turnover Comparison

For pedagogical reasons, the table below shows three measures of turnover. They use three different measures as the base.

- Invested Capital Turnover: More useful in analyzing ROIC. Uses Net Book Debt and Equity in the denominator.

- Gross Invested Capital Turnover: Uses gross debt rather than net debt. (Treats excess cash as part of the assets generating the sales.

- Asset Turnover: As used in ROE analysis. Book assets in denominator.

In [19]:
turnover_comps = roic.loc[['turnover (invested capital)']]
turnover_comps.loc['turnover (gross invested capital)'] = financials.loc['Sales (Net)'] / capital.loc[['debt','book equity']].sum().shift()
turnover_comps.loc['turnover (assets)'] = financials.loc['Sales (Net)'] / financials.loc['Assets - Total'].shift()
turnover_comps

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
turnover (invested capital),NaN,1.54,1.89,1.95,2.49,4.06,4.84,3.89,4.89,6.83,7.57,9.51,10.36,7.18,7.50,8.45,9.38,8.76,7.69,5.61,4.46,5.01
turnover (gross invested capital),NaN,1.22,1.36,1.41,1.83,2.74,2.59,2.40,2.23,2.04,2.06,2.27,2.04,1.45,1.30,1.59,1.17,1.06,1.06,1.17,1.38,1.95
turnover (assets),NaN,0.79,0.95,0.99,1.21,1.73,1.67,1.40,1.28,1.08,1.37,1.44,1.34,0.97,0.88,1.01,0.74,0.71,0.71,0.71,0.81,1.13


## NWC Analysis (2.3)

### The definition of NWC is

$\text{NWC} = \text{Adj. Current Assets} - \text{Adj. Current Liabilities}$

where the "Adjustments" refer to...

1. Remove current debt from current liabilities.

2. Remove current cash and liquid securities from current assets and replace it with required cash. (These will typically not offset.)

Thus,

$
\begin{align*}
\text{NWC} =& \underbrace{\left(\text{Current Assets} - \text{Current Cash} + \text{Required Cash}\right)}_{\text{Adj. Current Assets}}\\
& - \underbrace{\left(\text{Current Liabilities} - \text{Current Portion of Debt}\right)}_{\text{Adj. Current Liabilities}}
\end{align*}
$

### Model for Current Assets and Liabilities or Detailed Statements

With simplified financial statements, it is common to focus on the two main components of Current Assets and the one main component of Current Liabilities:

$
\begin{align*}
& \text{Adj. Current Assets} \; =\; \text{Accounts Receivables} + \text{Inventory}\\
& \text{Adj. Current Liabilities} \; =\; \text{Accounts Payables}
\end{align*}
$

For pedagogical reasons, the table below compares the two:
- stated NWC (which include all current assets and liabilities)
- simplified NWC (which include just the main components above, seen often in Harvard Case exhibits and other simplified settings.)

In [20]:
current_mod = pd.DataFrame({'adj. current assets': financials.loc['Receivables'] - financials.loc['Accounts Payable'] + financials.loc['Inventories - Total'] + cash.loc['required cash']}).T
current_mod.loc['adj. current liabilities'] = financials.loc['Accounts Payable']
current_mod.loc['nwc'] = current_mod.loc['adj. current assets'] - current_mod.loc['adj. current liabilities']
current_mod.loc['increase nwc'] = current_mod.loc['nwc'].diff()

current_stated = pd.DataFrame({'adj. current assets': financials.loc['Current Assets - Total'] - cash.loc['current cash'] + cash.loc['required cash']}).T
current_stated.loc['adj. current liabilities'] = financials.loc['Current Liabilities - Total'] - financials.loc['Debt in Current Liabilities']
current_stated.loc['nwc'] = current_stated.loc['adj. current assets'] - current_stated.loc['adj. current liabilities']
current_stated.loc['increase nwc'] = current_stated.loc['nwc'].diff()

current_vs = pd.concat([current_mod,current_stated],keys=['Simplified','Stated'])
current_vs.style.format("{:,.1f}")

Given that we have the full statements, we need not rely on the simplified model.

Rather, we use the full stated NWC.

In [21]:
USE_NWC_CURRENT = 'Stated'
current = current_vs.loc[USE_NWC_CURRENT]

### Cash Conversion Cycle

The Cash Conversion Cycle has not changed substantially since the time of the case.

It is marked by high payables days and negative CCC.

In [22]:
days = pd.DataFrame({'inventory days': 365 * financials.loc['Inventories - Total'] / financials.loc['Cost of Goods Sold']}).T 
days.loc['receivables days'] = 365 * financials.loc['Receivables'] / financials.loc['Sales (Net)']
days.loc['payables days'] = 365 * financials.loc['Accounts Payable'] / financials.loc['Cost of Goods Sold'] 

days.loc['cash conversion cycle'] = days.loc['inventory days'] + days.loc['receivables days'] - days.loc['payables days']

days.style.format("{:.1f}")

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
inventory days,2.1,1.0,4.1,4.7,6.3,6.2,7.3,8.1,8.9,6.6,9.9,4.5,3.4,6.4,7.4,6.6,6.4,13.5,9.4,10.0,9.4,11.9
receivables days,43.6,31.7,35.9,55.9,46.3,34.4,53.8,61.3,52.9,43.0,55.5,39.5,43.6,44.1,54.4,47.4,49.7,56.8,67.4,64.3,49.8,51.4
payables days,73.7,72.6,82.7,96.0,90.2,66.8,91.6,116.6,96.4,81.8,113.6,85.3,91.3,81.8,105.7,100.0,112.0,136.0,133.5,113.1,97.4,99.1
cash conversion cycle,-28.0,-39.9,-42.7,-35.5,-37.6,-26.2,-30.5,-47.2,-34.7,-32.1,-48.1,-41.3,-44.3,-31.2,-43.9,-46.0,-55.8,-65.7,-56.6,-38.8,-38.3,-35.8


## Cash Management (2.4)

Equity cashflow is

$\text{cashflow to EQUITY investors} = \text{dividends} + \text{share repurchases} - \text{share issuance}$

In [23]:
cf_equity = pd.DataFrame({'dividends': financials.loc['Cash Dividends']}).T
cf_equity.loc['share repurchases'] = financials.loc['Purchase of Common and Preferred Stock']
cf_equity.loc['equity distributions'] = cf_equity.sum()
cf_equity.loc['equity payout ratio'] = cf_equity.loc['equity distributions'] / profit.loc['nopat']
cf_equity

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
dividends,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"2,488.00","10,564.00","11,126.00","11,561.00","12,150.00","12,769.00","13,712.00","14,119.00","14,081.00","14,467.00"
share repurchases,116.00,0.00,0.00,26.00,0.00,0.00,355.00,3.00,0.00,82.00,406.00,520.00,"1,226.00","23,942.00","46,158.00","36,752.00","31,292.00","34,774.00","75,265.00","69,714.00","75,992.00","92,527.00"
equity distributions,116.00,0.00,0.00,26.00,0.00,0.00,355.00,3.00,0.00,82.00,406.00,520.00,"3,714.00","34,506.00","57,284.00","48,313.00","43,442.00","47,543.00","88,977.00","83,833.00","90,073.00","106,994.00"
equity payout ratio,0.37,-0.00,0.00,26.00,0.00,0.00,0.22,0.00,0.00,0.01,0.03,0.02,0.09,0.96,1.49,0.93,0.99,1.04,1.55,1.57,1.59,1.13


Debt cashflow is

$\text{cashflow to DEBT investors} = \text{interest expense} + \text{debt repayment} - \text{interest income} - \text{debt issuance}$

Note that financial statements may not be clear about interest income vs expense.

Furthermore, the amount on the Income Statement and Cashflow Statement may not be identical due to issues of FX exchanage, accounting, etc.

We use the Cashflow Statement amount when available. (For Apple, they are nearly identical.)

In [24]:
cf_debt = pd.DataFrame({'interest': financials.loc['Interest Paid Net']}).T
cf_debt.loc['debt retired'] = financials.loc['Long Term Debt - Reduction']
cf_debt.loc['debt issued'] = -financials.loc['Long Term Debt - Issuance']
cf_debt.loc['debt distributions'] = cf_debt.sum()
cf_debt

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
interest,10.00,20.00,20.00,20.00,10.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,339.00,514.00,"1,316.00","2,092.00","3,022.00","3,423.00","3,002.00","2,687.00"
debt retired,0.00,0.00,0.00,0.00,300.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"2,500.00","3,500.00","6,500.00","8,805.00","12,629.00","8,750.00"
debt issued,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,-0.00,"-16,896.00","-11,960.00","-27,114.00","-24,954.00","-28,662.00","-6,969.00","-6,963.00","-16,091.00","-20,393.00"
debt distributions,10.00,20.00,20.00,20.00,310.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"-16,896.00","-11,621.00","-26,600.00","-21,138.00","-23,070.00","2,553.00","5,265.00",-460.00,"-8,956.00"


In [25]:
cf_capital = pd.concat([cf_equity.loc['equity distributions'],cf_debt.loc['debt distributions']],axis=1).T
cf_capital
cf_capital.loc['capital distributions'] = cf_capital.sum()
cf_capital

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
equity distributions,116.00,0.00,0.00,26.00,0.00,0.00,355.00,3.00,0.00,82.00,406.00,520.00,"3,714.00","34,506.00","57,284.00","48,313.00","43,442.00","47,543.00","88,977.00","83,833.00","90,073.00","106,994.00"
debt distributions,10.00,20.00,20.00,20.00,310.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"-16,896.00","-11,621.00","-26,600.00","-21,138.00","-23,070.00","2,553.00","5,265.00",-460.00,"-8,956.00"
capital distributions,126.00,20.00,20.00,46.00,310.00,0.00,355.00,3.00,0.00,82.00,406.00,520.00,"3,714.00","17,610.00","45,663.00","21,713.00","22,304.00","24,473.00","91,530.00","89,098.00","89,613.00","98,038.00"


### 2013 Plan

As described in Case (A), Apple announced that 2013-2015
- \$10 bn in share repurchases
- \$35 bn in dividends

They did more than this in just 2013!

As described in Case (B), Apple announced
- \$60 bn in share repurchases over 2013-2015
- \$11 bn in annual dividends
- Distribute approximately \$100bn over 2013-2015

In actuality,
- dividends were almost exactly in line with this
- buybacks were \$105bn over 2013-2015--nearly double the plan!

## Investment Rate (2.5)

Note again that depreciation on the cashflow statement does not match depreciation on the income statement or balance sheet, for reasons discussed above.

In [26]:
invest = pd.DataFrame({'capex':financials.loc['Capital Expenditures']}).T
invest.loc['depreciation'] = -financials.loc['Depreciation and Amortization']
invest.loc['nwc investment'] = current.loc['increase nwc']
invest.loc['net investment'] = invest.sum(skipna=False)
invest.loc['investment rate'] = invest.loc['net investment'] / profit.loc['nopat']
invest

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
capex,107.00,232.00,174.00,164.00,176.00,260.00,657.00,735.00,"1,091.00","1,144.00","2,005.00","4,260.00","8,295.00","8,165.00","9,571.00","11,247.00","12,734.00","12,451.00","13,313.00","10,495.00","7,309.00","11,085.00"
depreciation,-84.00,-102.00,-118.00,-111.80,-149.50,-165.70,-209.80,-297.00,-436.00,-684.00,-932.00,"-1,822.00","-3,205.00","-6,757.00","-7,946.00","-10,500.00","-9,800.00","-9,400.00","-10,903.00","-12,547.00","-11,056.00","-11,284.00"
nwc investment,NaN,-134.00,82.00,127.00,97.00,610.00,877.00,543.00,"1,087.00","1,701.00","2,385.00","1,188.00","5,702.00","2,868.00","2,690.00",576.00,"-2,167.00","5,016.00","4,995.00","10,883.00","-1,778.00","11,724.00"
net investment,NaN,-4.00,138.00,179.20,123.50,704.30,"1,324.20",981.00,"1,742.00","2,161.00","3,458.00","3,626.00","10,792.00","4,276.00","4,315.00","1,323.00",767.00,"8,067.00","7,405.00","8,831.00","-5,525.00","11,525.00"
investment rate,NaN,0.01,5.75,179.20,0.51,0.60,0.82,0.34,0.41,0.27,0.25,0.14,0.26,0.12,0.11,0.03,0.02,0.18,0.13,0.17,-0.10,0.12


The Model for required cash is important, as it has a substantial impact on `nwc investment`. 

If `USE_CURRENT_CASH_AS_REQUIRED` is `True`, then this fluctuation leads to strange results.

To get better results, try an alternate model of required cash.
- Percent of revenue
- Percent of CAPEX
- Percent of SG&A

Accordingly, set `USE_CURRENT_CASH_AS_REQUIRED` as `False` to have required cash equal to $2\times\text{SG&A}$.

## Sustainable Growth Rate (2.6)

In [27]:
roe = financials.loc['Net Income (Loss)'] / financials.loc['Shareholders Equity - Total']

growth = pd.DataFrame({'sustainable growth (IC)': invest.loc['investment rate'] * roic.loc['roic']}).T
growth.loc['sustainable growth (E)'] = invest.loc['investment rate'] * roe

growth

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
sustainable growth (IC),NaN,-0.00,0.05,0.06,0.04,0.21,0.33,0.16,0.26,0.34,0.40,0.32,0.71,0.18,0.18,0.05,0.03,0.31,0.21,0.19,-0.09,0.16
sustainable growth (E),NaN,-0.00,0.09,2.93,0.03,0.11,0.16,0.08,0.10,0.07,0.07,0.05,0.09,0.04,0.04,0.01,0.01,0.06,0.07,0.10,-0.09,0.18


## Free-Cash-Flow

We can calculate FCF to all assets according to the usual formula:

$\text{FCF}_{A} = \text{EBIAT} - \text{Net Investment}$

or, in the case where there is no non-operating income, (or we're simplifying by ignoring it,)

$\text{FCF}_{A} = \text{NOPAT} - \text{Net Investment}$

Compare this to FCF compiled from the statement of cashflows.

In [28]:
cf_capital.loc['FCF to assets'] = profit.loc['nopat'] - invest.loc['net investment']
cf_capital

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
equity distributions,116.00,0.00,0.00,26.00,0.00,0.00,355.00,3.00,0.00,82.00,406.00,520.00,"3,714.00","34,506.00","57,284.00","48,313.00","43,442.00","47,543.00","88,977.00","83,833.00","90,073.00","106,994.00"
debt distributions,10.00,20.00,20.00,20.00,310.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"-16,896.00","-11,621.00","-26,600.00","-21,138.00","-23,070.00","2,553.00","5,265.00",-460.00,"-8,956.00"
capital distributions,126.00,20.00,20.00,46.00,310.00,0.00,355.00,3.00,0.00,82.00,406.00,520.00,"3,714.00","17,610.00","45,663.00","21,713.00","22,304.00","24,473.00","91,530.00","89,098.00","89,613.00","98,038.00"
FCF to assets,NaN,-314.00,-114.00,-178.20,118.50,465.70,299.80,"1,916.00","2,472.00","5,748.00","10,400.00","21,881.00","30,419.00","31,605.00","34,215.00","50,786.00","43,024.00","37,539.00","49,885.00","44,618.00","62,133.00","82,897.00"


The distributions to all capital differ substantially from the imputed FCF to all capital.

What accounts for the differrence?
- Apple did not pay out the FCF year-by-year
- Their distributions were much smaller in early years, as they accumulated excess cash.

### Valuation of FCF

Calculate the PV of the distribution to capital versus the FCF.
- Use a discount rate of 12%
- Do not consider a terminal value--just the value of these historic cashflows.

How close are the PVs?
- Actual cashflow distributed by Apple to investors (debt + equity)
- Modeled FCF

In [29]:
def npv(cf=0, rate=0, tv=0, price=0):
    
    cf_all = cf.copy()
    cf_all[-1] += tv
    pv = sum([cfi/(1+rate)**(i+1) for i, cfi in enumerate(cf_all)])
    npv = pv - price

    return npv

In [30]:
pv_comp = pd.DataFrame(data= [npv(cf_capital.loc['FCF to assets'].values[1:],.12), 
                        npv(cf_capital.loc['capital distributions'].values[1:],.12)],
                       index = ['FCF','Distributions'], columns=['PV'])

tv = cash.loc['excess cash'].values[-1:]
pv_comp.loc['Distributions adj'] = npv(cf_capital.loc['capital distributions'].values[1:],.12, tv=tv)
pv_comp

,PV
FCF,"83,167.38"
Distributions,"66,868.30"
Distributions adj,"76,378.06"


### Valuation of FCF is Close to Valuation of Actual Distributions
The PV of the historic distributions are \\$67bn, whereas the PV of the historic modeled FCF are \\$83bn.

But note that there is a difference: the historic distributions have allowed an accumulation of excess cash. So the PV of this excess cash should be added to the PV of the distributions.

With that adjustment, the PV of Distributions is \\$76bn, which is much closer.

While the modeled FCF will not lead to the exact same valuation as the actual distributions, it is a useful model--especially for forecasted financials, where we do not want to take a strong stand on when exactly cash will be distributed as dividends, repurchases, etc.